# Модель-пример

Импорт необходимых библиотек

In [ ]:
!git add .
!git commit -m "aug_models ready to train"
! git push

In [ ]:
import numpy as np
from PIL import Image
import tensorflow as tf
import torchvision as tv
from livelossplot import PlotLossesKeras

from core.datasets import get_ds
from core.make_answer import make_ans_file

Формируем аугментацию

In [ ]:
transformer = tv.transforms.Compose([
    tv.transforms.RandomAdjustSharpness(sharpness_factor=5, p=.4),
    tv.transforms.RandomHorizontalFlip(p=.5),
    tv.transforms.RandomPosterize(bits=3, p=.3),
    tv.transforms.RandomPerspective(distortion_scale=0.2, p=.5),
    tv.transforms.RandomRotation(degrees=(-10, 10)),
    tv.transforms.ColorJitter(brightness=.2, hue=0.2),
])

Создаем датасеты
```python
get_ds(
    transform=None,  `Аугументация`
    batch_size=512,  `Размер батча`
    val_size=0.03,  `Размер валидационного датасета, относительно data_train`
    back=1,  `На сколько папок наверх надо подняться, чтобы дойти до корневой`
)
```

In [ ]:
train_ds, val_ds, test_ds = get_ds(transform=transformer)

Пишем модель

In [ ]:
model_ex = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.AveragePooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.AveragePooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])
model_ex.summary()

Компиляция модели

In [ ]:
model_ex.compile(optimizer='adam', metrics=['accuracy'],
        loss=tf.keras.losses.SparseCategoricalCrossentropy())

Создаем чекпоинты

In [ ]:
checkpoint_filepath = '../checkpoints/checkpoint_ex.h5'
callback_ = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
)

Обучаем модель

In [ ]:
model_ex.fit(train_ds, validation_data=val_ds, epochs=100,
          callbacks=[PlotLossesKeras(), callback_], verbose=False)

Скачиваем модель из эпохи, которая показала лучшую метрику

In [ ]:
model_ex_best = tf.keras.models.load_model(checkpoint_filepath)
pred = model_ex_best.predict(test_ds / 255., verbose=False)

Формируем ответы в папку ans

In [ ]:
make_ans_file('models/answers/model_ex/checkpoint_ex.ans', pred)